In [4]:
n_exp = 10

algorithms = {
    #"old" : ["CSMOTE"],
    "new" : ["WEOB1", "improved_UOB"]#"ESOS_ELM","improved_OOB","OOB","UOB", "RebalanceStream","WEOB2"
}

datasets = {
    "real" : ["Elec","PAKDD", "KDDCup"],
    "art1" : ["sea_", "sine_","seaG_","sineG_"],
    "art2" : ["incremental","sudden", "periodic"]
}

directories = {
    "art1" : ["P(y)","P(x_y)","P(y_x)"],
    "res_art1" : ["P\\(y\\)","P\\(x_y\\)","P\\(y_x\\)"],
    "name_art1" : ["Py","Px_y","Py_x"],

    "art2" : ["appearing-minority","disappearing-minority","jitter","clusters-movement","appearing-clusters",
              "splitting-clusters", "borderline", "shapeshift", "minority-share"]
}


cnt = 0
options = [1,2,3,4]
test = open("QUERIES.txt","a")

for alg in algorithms["new"]:
    for di,d in enumerate(directories["art1"]):
        for o in options:
            for data in datasets["art1"]:
                name = f'{directories["name_art1"][di]}_{alg}_{data}{str(o)}_{str(10-o)}'
                query = 'from(bucket: "edv")\n  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)\n  |> filter(fn: (r) => r._measurement == "docker_container_mem")\n  |> filter(fn: (r) => r._field == "usage")\n  |> filter(fn: (r) => '
                for i in range(1,n_exp+1):
                    exp = f'r.container_name == "{directories["name_art1"][di]}_{alg}_{data}{str(o)}_{str(10-o)}_{str(i)}"'
                    if i > 1 and i <= n_exp:
                        exp = " or " + exp
                    query = query + exp
                test.write( query + f')\n  |> aggregateWindow(every: v.windowPeriod, fn: last)\n  |> last()\n  |> group(columns: ["_usage"])\n  |> mean()\n  |> yield(name: "{name}")\n\n')

# for alg in algorithms["new"]:
#     for di,d in enumerate(directories["art2"]):
#         for o in options:
#             for data in datasets["art2"]:
#                 name = f'{d}_{data}_{str(o)}_{alg}'
#                 query = 'from(bucket: "edv")\n|> range(start: v.timeRangeStart, stop: v.timeRangeStop)\n|> filter(fn: (r) => r._measurement == "docker_container_mem")\n|> filter(fn: (r) => r._field == "usage")\n|> filter(fn: (r) => '
#                 for i in range(1,n_exp+1):
#                     exp = f'r.container_name == "{d}_{data}_{str(o)}_{alg}_{str(i)}"'
#                     if i > 1 and i <= n_exp:
#                         exp = " or " + exp
#                     query = query + exp
#                 test.write( query + f')\n  |> aggregateWindow(every: v.windowPeriod, fn: last)\n  |> last()\n  |> group(columns: ["_usage"])\n  |> mean()\n  |> yield(name: "{name}")\n\n')
#
# for alg in algorithms["new"]:
#         for data in datasets["real"]:
#             name = f'{data}_{alg}'
#             query = 'from(bucket: "edv")\n  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)\n  |> filter(fn: (r) => r._measurement == "docker_container_mem")\n  |> filter(fn: (r) => r._field == "usage")\n  |> filter(fn: (r) => '
#             for i in range(1,n_exp+1):
#                 exp = f'r.container_name == "{data}_{alg}_{i}"'
#                 if i > 1 and i <= n_exp:
#                     exp = " or " + exp
#                 query = query + exp
#             test.write( query + f')\n  |> aggregateWindow(every: v.windowPeriod, fn: last)\n  |> last()\n  |> group(columns: ["_usage"])\n  |> mean()\n  |> yield(name: "{name}")\n\n')
#
#
#
test.close()
# print(cnt)